# Exercise 1

In [1]:
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip

#This will be doing most of the work
import networkx as nx

import sklearn #For generating some matrices
import pandas #For DataFrames
import numpy as np #For arrays
import matplotlib.pyplot as plt #For plotting
import seaborn #Makes the plots look nice
import scipy #Some stats
import nltk #a little language code
from IPython.display import Image #for pics

import pickle #if you want to save layouts
import os

%matplotlib inline

In [2]:
with open('text_by_dec.pickle', 'rb') as handle:
    decsDF = pickle.load(handle)

In [3]:
decsDF.head()

,Decision,Texts,tokenized_sents,normalized_sents
0,decisions/cp1/1-1.pdf,FCCC/CP/1995/7/Add.1EnglishPage 4I. DECISIONS...,"[[FCCC/], [CP/1995/7], [/Add.1EnglishPage], [4...","[[fccc/], [cp/1995/7], [/add.1englishpage], [4..."
1,decisions/cp1/1-2.pdf,FCCC/CP/1995/7/Add.1EnglishPage 7Decision 2/CP...,"[[FCCC/], [CP/1995/7], [/Add.1EnglishPage, 7De...","[[fccc/], [cp/1995/7], [/add.1englishpage, 7de..."
2,decisions/cp1/1-3.pdf,FCCC/CP/1995/7/Add.1EnglishPage 13Decision 3/C...,"[[FCCC/], [CP/1995/7/], [Add.1EnglishPage, 13D...","[[fccc/], [cp/1995/7/], [add.1englishpage, 13d..."
3,decisions/cp1/1-4.pdf,FCCC/CP/1995/7/Add.1EnglishPage 15Decision 4/C...,"[[FCCC/], [CP/1995/7, Add.1EnglishPage, 15Deci...","[[fccc/], [cp/1995/7, add.1englishpage, 15deci..."
4,decisions/cp1/1-5.pdf,FCCC/CP/1995/7/Add.1EnglishPage 18Decision 5/C...,"[[FCCC/], [CP/1995/7/], [Add.1EnglishPage, 18D...","[[fccc/], [cp/1995/7/], [add.1englishpage, 18d..."


In [13]:
decsDF.shape

(505, 4)

In [4]:
decsDF.loc(1)['normalized_sents'][0]

[['fccc/'],
 ['cp/1995/7'],
 ['/add.1englishpage'],
 ['4i'],
 ['decision', 'adopt', 'conference', 'partiesdecision'],
 ['/cp.1the'],
 ['berlin',
  'mandate',
  'review',
  'adequacy',
  'article',
  '4,paragraph',
  'b',
  'convention',
  'include',
  'proposalsrelated',
  'protocol',
  'decision',
  'follow',
  'upthe',
  'conference',
  'party',
  'session',
  'have',
  'review',
  'article',
  'paragraph',
  '2(a',
  'b',
  'unite',
  'nation',
  'frameworkconvention',
  'climate',
  'change',
  'andhaving'],
 ['conclude',
  'subparagraph',
  'adequate',
  'agree',
  'begin',
  'process',
  'enable',
  'appropriate',
  'action',
  'period',
  'beyond2000',
  'include',
  'strengthen',
  'commitment',
  'party',
  'include',
  'annex'],
 ['tothe',
  'convention',
  'annex',
  'party',
  'article',
  'paragraph',
  '2(a',
  'b',
  'adoptionof',
  'protocol',
  'legal',
  'instrument:1.the',
  'process',
  'shall',
  'guide'],
 ['inter', 'alia', 'follow'],
 ['i(a)the',
  'provision',
 

In [11]:
import sklearn.feature_extraction
def tokenize(text):
    tokenlist = lucem_illud_2020.word_tokenize(text)
    normalized = lucem_illud_2020.normalizeTokens(tokenlist)
    return normalized

#Now normalize the texts
vectorizer = sklearn.feature_extraction.text.CountVectorizer(tokenizer = tokenize)
vects_incidence = vectorizer.fit_transform(decsDF['Texts'])

In [ ]:
#Now begin to create network by word occurance
g_2mode = nx.Graph()

#define all the nodes
g_2mode.add_nodes_from((vectorizer.get_feature_names()[i] for i in range(vects_incidence.shape[1])), bipartite = 'word')
g_2mode.add_nodes_from(range(vects_incidence.shape[0]), bipartite = 'doc')

#add all the edges
g_2mode.add_edges_from(((d, vectorizer.get_feature_names()[w], {'weight' : vects_incidence[d, w]}) for d, w in zip(*vects_incidence.nonzero())))



In [ ]:
def contractNetwork(g, targetType):
    g_mono = nx.Graph()
    g_mono.add_nodes_from(((n, d) for n, d in g_2mode.nodes(data = True) if d['bipartite'] == targetType))
    
    for n_outside in (n for n, d in g_2mode.nodes(data = True) if d['bipartite'] != targetType):
        neighbors = list((n for n in g.neighbors(n_outside) if g.nodes[n]['bipartite'] == targetType))
        for i, n1 in enumerate(neighbors):
            for n2 in neighbors[i+1:]:
                try:
                    g_mono.edges[n1, n2]['weight'] += 1
                except KeyError:
                    g_mono.add_edge(n1, n2, weight = 1)
    return g_mono

In [ ]:
gDoc = contractNetwork(g_2mode, 'doc')
print(nx.info(gDoc))

In [ ]:
layout = nx.spring_layout(gChap, k = 1/3, weight='weight', iterations= 50)
nx.draw(gDoc, pos = layout, labels = {n:n for n in gDoc.nodes()}) #Getting labels is a bit annoying